In [1]:
import diverse_yeast_tools as dyt
import os
import pandas as pd
from collections import Counter

base_dir = os.path.normpath('G:/My Drive/Crick_LMS/projects/diverse_yeasts/alphafold/') + os.sep

In [2]:
species_set = pd.read_csv(os.path.normpath(base_dir + 'selected_proteins/species_selection/species_selection.csv'), index_col = 0)
species_set_filt = species_set.loc[species_set['Load']=='Y']

phenotype_data = pd.read_csv(os.path.normpath(base_dir + 'selected_proteins/species_selection/lu_et_al_substrate_utilization.csv'))



In [3]:
#All but S. pombe have matches in the species data 
set(list(species_set_filt['jacobs_name'])) - set(list(phenotype_data.columns)) 

{'Schizosaccharomyces_pombe'}

In [4]:
species_jacobs_names = list(set(list(phenotype_data.columns)) & set(list(species_set_filt['jacobs_name'])))

In [5]:
information_columns = ['Original names', 'Name in the model', 'Aerobic/Anaerobic', 'Source']

phenotype_data_subset_raw = phenotype_data.loc[:,species_jacobs_names]



In [6]:
# Could use this to filter species.  Ogataea_parapolymorpha has 61 blanks.

species_blanks = {}

for spec in species_jacobs_names:
    blank_counts = Counter(phenotype_data_subset_raw.loc[:,spec])
    species_blanks[spec] = blank_counts['n']
                          
species_blanks

{'Zygosaccharomyces_rouxii': 20,
 'Komagataella_pastoris': 16,
 'Candida_tropicalis': 11,
 'Torulaspora_delbrueckii': 14,
 'Pachysolen_tannophilus': 16,
 'yHMPu5000034957_Hanseniaspora_osmophila': 18,
 'Kluyveromyces_marxianus': 12,
 'Yarrowia_lipolytica': 16,
 'Candida_albicans': 11,
 'Vanderwaltozyma_polyspora': 12,
 'Candida_apicola': 10,
 'Lipomyces_starkeyi': 17,
 'yHMPu5000034604_Sporopachydermia_lactativora': 11,
 'Lachancea_thermotolerans': 12,
 'Kazachstania_naganishii': 12,
 'Tortispora_caseinolytica': 12,
 'Eremothecium_gossypii': 17,
 'Geotrichum_candidum': 20,
 'Cyberlindnera_jadinii': 16,
 'Saccharomyces_cerevisiae': 12,
 'Alloascoidea_hylecoeti': 15,
 'Debaryomyces_hansenii': 15,
 'Ogataea_parapolymorpha': 61,
 'Wickerhamomyces_anomalus': 18,
 'Kluyveromyces_lactis': 13,
 'Ascoidea_rubescens': 15}

In [8]:
phenotype_data_subset_raw

,Zygosaccharomyces_rouxii,Komagataella_pastoris,Candida_tropicalis,Torulaspora_delbrueckii,Pachysolen_tannophilus,yHMPu5000034957_Hanseniaspora_osmophila,Kluyveromyces_marxianus,Yarrowia_lipolytica,Candida_albicans,Vanderwaltozyma_polyspora,...,Eremothecium_gossypii,Geotrichum_candidum,Cyberlindnera_jadinii,Saccharomyces_cerevisiae,Alloascoidea_hylecoeti,Debaryomyces_hansenii,Ogataea_parapolymorpha,Wickerhamomyces_anomalus,Kluyveromyces_lactis,Ascoidea_rubescens
0,1,1,1,1,1,1,1,0,1,1,...,0,v,1,1,0,v,n,1,1,0
1,0,0,1,v,0,0,1,0,v,1,...,0,v,0,v,0,v,n,v,1,0
2,v,0,v,v,0,0,1,0,v,1,...,0,0,1,1,0,v,n,1,v,0
3,1,0,1,v,0,0,0,0,1,0,...,0,0,0,1,0,v,n,v,v,0
4,0,0,0,0,0,0,v,0,0,0,...,0,0,0,0,0,0,n,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,0,n,n,n,n,0,n,n,n,n,...,n,0,n,v,0,0,n,n,n,0
71,0,n,n,n,n,0,n,n,n,n,...,n,0,n,n,0,0,n,n,n,0
72,6,8,9,6,8,6,6,9,9,6,...,6,n,7,6,n,9,n,7,6,n
73,41.2,41.3,34.7,43.4,37.8,40.1,41.3,49.9,35,35.3,...,n,42,44.7,39.9,n,36.8,n,36.5,40.3,n


In [30]:
phenotype_lookup = {}
ind_lookup = {}
for jj, name in enumerate(phenotype_data['Original names']):
    phenotype_lookup[jj] = name
    ind_lookup[name] = jj

In [38]:
n_blank_threshold = 5  #Number of blanks allowed for the data to be kept
conditions_to_keep = []
conditions_to_remove = {'CoQ (Main component)', 'Mol % G+C (Ave.)'}
for ind, row in phenotype_data_subset_raw.iterrows():
    row_counts = Counter(row)
    #print(str(ind) + ' ' + str(row_counts['n']))
    
    if phenotype_lookup[ind] in conditions_to_remove:
        print('Manually Removed becuase of non-binary data: ' + phenotype_lookup[ind])
    else: 
        
        #Filter out rows with too much missing data
        if row_counts['n'] <= n_blank_threshold:
            #Filter out uninformative rows - no 0's or no 1's
            if (row_counts['0']>0) & (row_counts['1']>0): 
                conditions_to_keep.append(ind)
            else: 
                print('Uninformative condition removed: ' + phenotype_lookup[ind])
        else: 
            print('Condition removed because more than ' + str(n_blank_threshold) + ' species had no data: ' + phenotype_lookup[ind])
        
        
    
            
    

Uninformative condition removed: Glucose
Uninformative condition removed: Galactitol
Condition removed because more than 5 species had no data: N-Acetyl-D-glucosamine
Condition removed because more than 5 species had no data: Hexadecane
Condition removed because more than 5 species had no data: Nitrite
Condition removed because more than 5 species had no data: 2-Keto-D-gluconate
Condition removed because more than 5 species had no data: 5-Keto-D-gluconate
Condition removed because more than 5 species had no data: Saccharate
Condition removed because more than 5 species had no data: Xylitol
Condition removed because more than 5 species had no data: L-Arabinitol
Condition removed because more than 5 species had no data: Arbutin
Condition removed because more than 5 species had no data: Propane 1,2 diol
Condition removed because more than 5 species had no data: Butane 2,3 diol
Condition removed because more than 5 species had no data: Cadaverine
Condition removed because more than 5 speci

In [39]:
phenotype_data_subset_conds = phenotype_data.loc[:,information_columns + species_jacobs_names]
phenotype_data_subset_conds.loc[conditions_to_keep,:].to_csv(base_dir+ 'selected_proteins/species_selection/lu_et_al_substrate_utilization_subset.csv')